**Resources**
<!-- {{< youtube akZ8JJ4gGLs >}} -->
<!-- {{< tweet 1266579585417613312 >}} -->
[Smoothstep](https://smoothstep.io/)

- Breadth First Search
- Dijkstra's algorithm/Fast Marching Method for solving the Eikonal equation?
- BFS is generalized as Disjkstra, which is generalized as Fast Marching, then as Ordered Upwind method, then as Anisotropic Fast Marching

- Hex mazes
- Bellman-Ford Algorithm
- Fast Marching Algorithm (FMM), eikonal equation
- Art: https://troika.uk.com/work/troika-labyrinth/
- https://en.wikipedia.org/wiki/Lee_algorithm
- [Procedural Content Generation: Mazes](http://pcg.wikidot.com/pcg-algorithm:maze)
- [Wikipedia Maze Generation Algorithms](https://en.wikipedia.org/wiki/Maze_generation_algorithm)


Solving a Maze
- https://en.wikipedia.org/wiki/Maze-solving_algorithm


Per wikipedia:
> The best-known rule for traversing mazes is the wall follower, also known as either the left-hand rule or the right-hand rule. If the maze is simply connected, that is, all its walls are connected together or to the maze's outer boundary, then by keeping one hand in contact with one wall of the maze the solver is guaranteed not to get lost and will reach a different exit if there is one; otherwise, the algorithm will return to the entrance having traversed every corridor next to that connected section of walls at least once. The algorithm is a depth-first in-order tree traversal.

For autonomous agents I perhaps one of these will be suitable.
- [Maze Routing Algorithm](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Maze-routing_algorithm)
- [Shortest Path Algorithms](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Shortest_path_algorithm)


What do I really want to do here?
- Ultimately, I want to get back to working on the game. 
For NPCs:
  - Find the shortest distance between to points. Example enter/exit the maze.
  - Follow another character in the maze.
  - Search for an item in the maze.
  - animate moving a maze explorer
  - draw out the solution and possibly rejected solutions.
  - The AI should only have knowledge of the rooms its visited and the room it's
    currently in. Is that true though?

Questions
- What algorithm does Godot use?
  A* and 

Next Steps
- Try doing a simple animation using iPyCanvas. 
- Need a way to make an animated gif from the animation if I'm going to be able
  to put it on the blog.
- Agent that just walks the maze randomly.
- Agent that uses an algorithm for traversing.
- Multiple agents interacting (chasing, hiding)
- It may make sense to generate the same maze every time. Perhaps seed the random number generator. 

In [1]:
# Load python code.
%load_ext autoreload
%autoreload 2

In [2]:
# All imports
from ipycanvas import Canvas,  hold_canvas
import random
import time
from typing import List

from generation.structures import Point
from generation.maze import Maze
from generation.generators.random_backtracer import generate_maze_walls
from generation.npc import Agent
from generation.renderers.wall_drawer import draw_maze, animate_drawing_by_rooms
from generation.renderers.agents_renderer import draw_agents
from generation.direction import Direction, DIR_ORIENTATION, Orientation

In [3]:
def find_next_direction(agent: Agent, possible_directions: list[Direction]) -> Direction:
  """The agent is facing a direction. I think it should continue in the same direction
  if possible. If not it should try right, then left. Finally, backtrack."""
  current_orientation: dict[Orientation, Direction] = DIR_ORIENTATION[agent.facing]
  if agent.facing in possible_directions:
    next_direction = agent.facing
  elif current_orientation[Orientation.RIGHT] in possible_directions: # Is there a door to the right?
    next_direction = current_orientation[Orientation.RIGHT]
  elif current_orientation[Orientation.LEFT]  in possible_directions: # Is there a door to the left?
    next_direction = current_orientation[Orientation.LEFT]
  elif current_orientation[Orientation.BEHIND] in possible_directions: # Go back the way we came?
    next_direction = current_orientation[Orientation.BEHIND]
  else:
    # This shouldn't be possible
    print('We\'re walled in!')
    print(f'Agent Facing: {agent.facing}')
    print(f'Possible Directions: {possible_directions}')
    print(f'Agent Orientation: {current_orientation}')
    raise Exception("We\'re walled in! No possible doors found.")
  return next_direction

# Traversal
def random_walk(agent: Agent, maze: Maze):
  """
  Given an agent, have them randomly choose a room to go to next, then move.
  """
  # 1. Get the current room the agent is in.
  current_room = maze.cell(agent.location)

  # 2. Find all the walls that have doors in that room.
  possible_directions: List[Direction] = current_room.open_sides()

  # 3. Find the next direction to go.
  next_direction = find_next_direction(agent, possible_directions)

  # 4. Find the location of the room the open door connects to.
  next_location: Point = maze.find_adjacent_neighbor(next_direction, agent.location)

  # 5. Move the agent to the next room.
  agent.face(next_direction)
  agent.move_to(next_location)

  # TODO. Don't pick the door that you came through unless it's the only option.
  # Consider using a Stack to track every room the agent has visited.
  # What should happen if there is a dead end?

In [9]:
# 1000/125 = 8 FPS
SLEEP_TIME_SEC:float = 0.125

# Generate a maze.
maze: Maze = Maze(20, 20)
generate_maze_walls(maze)
canvas = Canvas(width=800, height=400)
display(canvas)

# Place the rouge in the center of the maze
rogue = Agent('blue')
rogue.move_to(Point(maze.width/2, maze.height/2))
rogue.face(Direction.SOUTH)

agents = []
agents.append(rogue)

# Initial Render
draw_maze(maze, canvas)
draw_agents(agents, canvas)

for i in range(200):
  # canvas.fill_style = 'white'
  # canvas.fill_rect(0,0, canvas.width, canvas.height)
  for agent in agents:
    random_walk(agent, maze)
  draw_agents(agents, canvas)
  time.sleep(SLEEP_TIME_SEC)

Exit cell: Point(x=17, y=19)


Canvas(height=400, width=800)

KeyboardInterrupt: 

In [5]:
# Let's skip this for right now when running all.

from ipywidgets import Output
from ipycanvas import Canvas, hold_canvas
from threading import Thread
import numpy as np
import time
from IPython.display import display, Image
from moviepy.editor import ImageSequenceClip

out = Output()

# 1000/125 = 8 FPS
SLEEP_TIME_SEC:float = 0.125

class Renderer:
    def __init__(self, canvas):
        self.frames = []
        self.canvas = canvas
        self.canvas.observe(self.get_array, 'image_data')
        self.setup_canvas()
        
    @out.capture()
    def get_array(self, *args, **kwargs):
        print("get_array called")
        arr = self.canvas.get_image_data()
        self.frames.append(arr)
        
    def setup_canvas(self):
        self.canvas.fill_style = 'white'
        self.canvas.fill_rect(0,0,self.canvas.width, self.canvas.height)
        
    def render(self, x):
        self.canvas.stroke_line(50, 10+(x*5), 150, 10+(x*5))
        
class MyThread(Thread):
    def __init__(self, renderer):
        super(MyThread, self).__init__()
        self.renderer = renderer
    
    def run(self):
        for i in range(8):
            self.renderer.render(i)
            time.sleep(SLEEP_TIME_SEC)
        self.on_complete()    

    def on_complete(self):
        self.save_gif()
        
    def save_gif(self):
        # save it as a gif
        print(f"Number of frames: {len(self.renderer.frames)}")
        clip = ImageSequenceClip(list(self.renderer.frames), fps=8)
        clip.write_gif('test.gif', fps=8)
        Image('test.gif')

# display(out)
# canvas = Canvas(width=200, height=200, sync_image_data=True)
# renderer = Renderer(canvas)
# thread = MyThread(renderer)

# display(canvas)
# thread.start()

In [6]:
# The Main Entry Point
"""
SAVE_ANIMATED_GIF = False

display(out)
canvas = Canvas(width=200, height=200, sync_image_data=True)
renderer = Renderer(canvas, SAVE_ANIMATED_GIF)
thread = RenderingThread(renderer, SAVE_ANIMATED_GIF)

display(canvas)
thread.start()
"""

'\nSAVE_ANIMATED_GIF = False\n\ndisplay(out)\ncanvas = Canvas(width=200, height=200, sync_image_data=True)\nrenderer = Renderer(canvas, SAVE_ANIMATED_GIF)\nthread = RenderingThread(renderer, SAVE_ANIMATED_GIF)\n\ndisplay(canvas)\nthread.start()\n'